In [ ]:
[root@ip-172-31-8-221 docker-image]# ll
total 20
-r-------- 1 root root 1679 Jul  1 13:34 ansible.pem
-rw-r--r-- 1 root root   57 Jul  1 21:27 cred.vars
-rw-r--r-- 1 root root 2293 Jul  1 21:29 docker-image.yml
-rw-r--r-- 1 root root  107 Jul  1 21:14 hosts
-rw-r--r-- 1 root root   93 Jul  1 18:32 image.vars
[root@ip-172-31-8-221 docker-image]# cat image.vars 
--- 
git_url: "https://github.com/jinumona/devops-flask.git"
git_dir: "/home/ec2-user/image"


### Installing  docker and git cloing

In [ ]:
---
- name: "Creating docker Image on Amazon Linux"
  hosts: amazon
  become: yes
  vars_files:
    - image.vars 

  tasks:
    - name: "Installing Docker,pip,git on Amazon Linux"
      yum:
        name: 
          - docker
          - python-pip
          - git
        state: present
      register: docker_status



    - name: "Installing python-docker connector/module"
      pip:
        name: 
          - docker-py
        state: present

    - name: "Adding ec2-user to docker group"
      user:
        append: yes
        groups: docker
        name: ec2-user

    - name: "Restarting/enabling docker"
      when: docker_status.changed == true 
      service:
        name: docker 
        state: restarted
        enabled: yes


    - name: "cloning Docker Files"
      git:
        dest: "{{ git_dir }}"
        repo: "{{ git_url }}"
      register: git_status

    - name: "Debug git"
      debug:
        var: git_status 

In [ ]:
# ansible-playbook -i hosts docker-image.yml 

PLAY [Creating docker Image on Amazon Linux] **************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
[WARNING]: Platform linux on host 172.31.8.180 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.8.180]

TASK [Installing Docker,pip,git on Amazon Linux] **********************************************************************************************************
changed: [172.31.8.180]

TASK [Installing python-docker connector/module] **********************************************************************************************************
changed: [172.31.8.180]

TASK [Adding ec2-user to docker group] ********************************************************************************************************************
changed: [172.31.8.180]

TASK [Restarting/enabling docker] *************************************************************************************************************************
changed: [172.31.8.180]

TASK [cloning Docker Files] *******************************************************************************************************************************
changed: [172.31.8.180]

TASK [Debug git] ******************************************************************************************************************************************
ok: [172.31.8.180] => {
    "git_status": {
        "after": "0f16c63a136046d3fd34b44c752299c1226656b2", 
        "before": null, 
        "changed": true, 
        "failed": false
    }
}

PLAY RECAP ************************************************************************************************************************************************
172.31.8.180               : ok=7    changed=5    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0 

### Building an image with tags if git status chnaged

In [ ]:
# cat docker-image.yml 
---
- name: "Creating docker Image on Amazon Linux"
  hosts: amazon
  become: yes
  vars_files:
    - image.vars 

  tasks:
    - name: "Installing Docker,pip,git on Amazon Linux"
      yum:
        name: 
          - docker
          - python-pip
          - git
        state: present
      register: docker_status



    - name: "Installing python-docker connector/module"
      pip:
        name: 
          - docker-py
        state: present

    - name: "Adding ec2-user to docker group"
      user:
        append: yes
        groups: docker
        name: ec2-user

    - name: "Restarting/enabling docker"
      when: docker_status.changed == true 
      service:
        name: docker 
        state: restarted
        enabled: yes


    - name: "cloning Git Docker Files"
      git:
        dest: "{{ git_dir }}"
        repo: "{{ git_url }}"
      register: git_status

    - name: "Debug git"
      debug:
        var: git_status 
    

    - name: "Building Docker Image"
      when: git_status.changed == true
      community.docker.docker_image:
        name: jinumona/devops-flaskapp
        build:
          path: "{{ git_dir }}"
        source: build 
        state: present
        tag: "{{ item }}"
        force_source: yes
        force_tag: yes
      with_items:
        - "{{ git_status.after }}"
        -  latest


In [ ]:
---
- name: "Creating docker Image on Amazon Linux"
  hosts: amazon
  become: yes
  vars_files:
    - image.vars 
    - cred.vars

  tasks:
    - name: "Installing Docker,pip,git on Amazon Linux"
      yum:
        name: 
          - docker
          - python-pip
          - git
        state: present
      register: docker_status



    - name: "Installing python-docker connector/module"
      pip:
        name: 
          - docker-py
        state: present

    - name: "Adding ec2-user to docker group"
      user:
        append: yes
        groups: docker
        name: ec2-user

    - name: "Restarting/enabling docker"
      when: docker_status.changed == true 
      service:
        name: docker 
        state: restarted
        enabled: yes


    - name: "cloning Git Docker Files"
      git:
        dest: "{{ git_dir }}"
        repo: "{{ git_url }}"
      register: git_status

    - name: "Debug git"
      debug:
        var: git_status 
    

    - name: "Building Docker Image"
      when: git_status.changed == true
      community.docker.docker_image:
        name: jinumona/devops-flaskapp
        build:
          path: "{{ git_dir }}"
        source: build 
        state: present
        tag: "{{ item }}"
        force_source: yes
        force_tag: yes
      with_items:
        - "{{ git_status.after }}"
        -  latest

    - name: "Logging to Dockerhub...."
      when: git_status.changed == true
      community.docker.docker_login:
        password: "{{ hub_password }}"
        username: "{{ hub_username }}"


    - name: "Pushing the latest version changed Dockerimage to Hub"
      when: git_status.changed == true
      community.docker.docker_image:
        name: "jinumona/devops-flaskapp:{{ item }}"
        push: yes
        source: local
        force_tag: yes
      with_items: 
        - "{{ git_status.after }}"
        - latest

      

    - name: "Logout from Hub"
      when: git_status.changed == true
      community.docker.docker_login:
        state: absent


    
    - name: "removing the created local images"
      when: git_status.changed == true
      community.docker.docker_image:
        name: "jinumona/devops-flaskapp:{{ item }}"
        state: absent
      with_items: 
        - "{{ git_status.after }}"
        - latest

In [ ]:
#on clinet machine

# docker image ls
REPOSITORY                 TAG                                        IMAGE ID       CREATED         SIZE
jinumona/devops-flaskapp   42f0cd4fd3efd4f7bfe943f91edbb084d91f233e   f1fac8806850   3 minutes ago   62.8MB
jinumona/devops-flaskapp   latest                                     f1fac8806850   3 minutes ago   62.8MB
alpine                     3.8                                        c8bccc0af957   2 years ago     4.41MB

#### version chnaged

In [ ]:
# ansible-playbook -i hosts docker-image.yml 

PLAY [Creating docker Image on Amazon Linux] **************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
[WARNING]: Platform linux on host 172.31.8.180 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.8.180]

TASK [Installing Docker,pip,git on Amazon Linux] **********************************************************************************************************
ok: [172.31.8.180]

TASK [Installing python-docker connector/module] **********************************************************************************************************
ok: [172.31.8.180]

TASK [Adding ec2-user to docker group] ********************************************************************************************************************
ok: [172.31.8.180]

TASK [Restarting/enabling docker] *************************************************************************************************************************
skipping: [172.31.8.180]

TASK [cloning Git Docker Files] ***************************************************************************************************************************
changed: [172.31.8.180]

TASK [Debug git] ******************************************************************************************************************************************
ok: [172.31.8.180] => {
    "git_status": {
        "after": "70fd22bd7697f056b15ffefdb7b795576493b287", 
        "before": "a7600098d11f80d5af172cd989024a4934e0becb", 
        "changed": true, 
        "failed": false, 
        "remote_url_changed": false
    }
}

TASK [Building Docker Image] ******************************************************************************************************************************
changed: [172.31.8.180] => (item=70fd22bd7697f056b15ffefdb7b795576493b287)
changed: [172.31.8.180] => (item=latest)

PLAY RECAP ************************************************************************************************************************************************
172.31.8.180               : ok=7    changed=2    unreachable=0    failed=0    skipped=1    rescued=0    ignored=0

In [ ]:
# docker image ls 
REPOSITORY                 TAG                                        IMAGE ID       CREATED          SIZE
jinumona/devops-flaskapp   70fd22bd7697f056b15ffefdb7b795576493b287   24057061ff4a   50 seconds ago   62.8MB
jinumona/devops-flaskapp   latest                                     24057061ff4a   50 seconds ago   62.8MB
jinumona/devops-flaskapp   42f0cd4fd3efd4f7bfe943f91edbb084d91f233e   f1fac8806850   14 minutes ago   62.8MB
alpine                     3.8                                        c8bccc0af957   2 years ago      4.41MB

### Login to docker hub, pushing to hub,logout from hub, removing local images created

In [ ]:
# cat docker-image.yml 

---
- name: "Creating docker Image on Amazon Linux"
  hosts: amazon
  become: yes
  vars_files:
    - image.vars 
    - cred.vars

  tasks:
    - name: "Installing Docker,pip,git on Amazon Linux"
      yum:
        name: 
          - docker
          - python-pip
          - git
        state: present
      register: docker_status



    - name: "Installing python-docker connector/module"
      pip:
        name: 
          - docker-py
        state: present

    - name: "Adding ec2-user to docker group"
      user:
        append: yes
        groups: docker
        name: ec2-user

    - name: "Restarting/enabling docker"
      when: docker_status.changed == true 
      service:
        name: docker 
        state: restarted
        enabled: yes


    - name: "cloning Git Docker Files"
      git:
        dest: "{{ git_dir }}"
        repo: "{{ git_url }}"
      register: git_status

    - name: "Debug git"
      debug:
        var: git_status 
    

    - name: "Building Docker Image"
      when: git_status.changed == true
      community.docker.docker_image:
        name: jinumona/devops-flaskapp
        build:
          path: "{{ git_dir }}"
        source: build 
        state: present
        tag: "{{ item }}"
        force_source: yes
        force_tag: yes
      with_items:
        - "{{ git_status.after }}"
        -  latest

    - name: "Logging to Dockerhub...."
      when: git_status.changed == true
      community.docker.docker_login:
        password: "{{ hub_password }}"
        username: "{{ hub_username }}"


    - name: "Pushing the latest version changed Dockerimage to Hub"
      when: git_status.changed == true
      community.docker.docker_image:
        name: "jinumona/devops-flaskapp:{{ item }}"
        push: yes
        source: local
        force_tag: yes
      with_items: 
        - "{{ git_status.after }}"
        - latest

      

    - name: "Logout from Hub"
      when: git_status.changed == true
      community.docker.docker_login:
        state: absent


    
    - name: "removing the created local images"
      when: git_status.changed == true
      community.docker.docker_image:
        name: "jinumona/devops-flaskapp:{{ item }}"
        state: absent
      with_items: 
        - "{{ git_status.after }}"
        - latest


In [ ]:
# ansible-playbook -i hosts docker-image.yml 

PLAY [Creating docker Image on Amazon Linux] **************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
[WARNING]: Platform linux on host 172.31.14.65 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.14.65]

TASK [Installing Docker,pip,git on Amazon Linux] **********************************************************************************************************
ok: [172.31.14.65]

TASK [Installing python-docker connector/module] **********************************************************************************************************
ok: [172.31.14.65]

TASK [Adding ec2-user to docker group] ********************************************************************************************************************
ok: [172.31.14.65]

TASK [Restarting/enabling docker] *************************************************************************************************************************
skipping: [172.31.14.65]

TASK [cloning Git Docker Files] ***************************************************************************************************************************
changed: [172.31.14.65]

TASK [Debug git] ******************************************************************************************************************************************
ok: [172.31.14.65] => {
    "git_status": {
        "after": "a8b559f11fe29c6d9f40d74fa0cdb581117e0fac", 
        "before": "47ec40ff4a825551097bb273463e66eb848f517e", 
        "changed": true, 
        "failed": false, 
        "remote_url_changed": false
    }
}

TASK [Building Docker Image] ******************************************************************************************************************************
changed: [172.31.14.65] => (item=a8b559f11fe29c6d9f40d74fa0cdb581117e0fac)
changed: [172.31.14.65] => (item=latest)

TASK [Logging to Dockerhub....] ***************************************************************************************************************************
changed: [172.31.14.65]

TASK [Pushing the latest version changed Dockerimage to Hub] **********************************************************************************************
changed: [172.31.14.65] => (item=a8b559f11fe29c6d9f40d74fa0cdb581117e0fac)
ok: [172.31.14.65] => (item=latest)

TASK [Logout from Hub] ************************************************************************************************************************************
changed: [172.31.14.65]

TASK [removing the created local images] ******************************************************************************************************************
changed: [172.31.14.65] => (item=a8b559f11fe29c6d9f40d74fa0cdb581117e0fac)
changed: [172.31.14.65] => (item=latest)

PLAY RECAP ************************************************************************************************************************************************
172.31.14.65               : ok=11   changed=6    unreachable=0    failed=0    skipped=1    rescued=0    ignored=0 

## ansible-vault

In [ ]:
# ansible-vault encrypt cred.vars 
New Vault password: 
Confirm New Vault password: 
Encryption successful
[root@ip-172-31-8-221 docker-image]# cat cred.vars 
$ANSIBLE_VAULT;1.1;AES256
63326165666664323233633538396166653466393033356164393839366136323461373361376232
6430376431363137336434343239306438393830633835620a616234656637326333353834646138

In [ ]:
# ansible-playbook -i hosts docker-image.yml 
ERROR! Attempting to decrypt but no vault secrets found


In [ ]:
# ansible-playbook -i hosts docker-image.yml --ask-vault-pass
Vault password: 

PLAY [Creating docker Image on Amazon Linux] **************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
[WARNING]: Platform linux on host 172.31.14.65 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.14.65]

TASK [Installing Docker,pip,git on Amazon Linux] **********************************************************************************************************
ok: [172.31.14.65]

TASK [Installing python-docker connector/module] **********************************************************************************************************
ok: [172.31.14.65]

TASK [Adding ec2-user to docker group] ********************************************************************************************************************
ok: [172.31.14.65]

TASK [Restarting/enabling docker] *************************************************************************************************************************
skipping: [172.31.14.65]

TASK [cloning Git Docker Files] ***************************************************************************************************************************
ok: [172.31.14.65]

TASK [Debug git] ******************************************************************************************************************************************
ok: [172.31.14.65] => {
    "git_status": {
        "after": "a8b559f11fe29c6d9f40d74fa0cdb581117e0fac", 
        "before": "a8b559f11fe29c6d9f40d74fa0cdb581117e0fac", 
        "changed": false, 
        "failed": false, 
        "remote_url_changed": false
    }
}

TASK [Building Docker Image] ******************************************************************************************************************************
skipping: [172.31.14.65] => (item=a8b559f11fe29c6d9f40d74fa0cdb581117e0fac) 
skipping: [172.31.14.65] => (item=latest) 

TASK [Logging to Dockerhub....] ***************************************************************************************************************************
skipping: [172.31.14.65]

TASK [Pushing the latest version changed Dockerimage to Hub] **********************************************************************************************
skipping: [172.31.14.65] => (item=a8b559f11fe29c6d9f40d74fa0cdb581117e0fac) 
skipping: [172.31.14.65] => (item=latest) 

TASK [Logout from Hub] ************************************************************************************************************************************
skipping: [172.31.14.65]

TASK [removing the created local images] ******************************************************************************************************************
skipping: [172.31.14.65] => (item=a8b559f11fe29c6d9f40d74fa0cdb581117e0fac) 
skipping: [172.31.14.65] => (item=latest) 

PLAY RECAP ************************************************************************************************************************************************
172.31.14.65               : ok=6    changed=0    unreachable=0    failed=0    skipped=6    rescued=0    ignored=0 

In [ ]:
]# ansible-vault decrypt cred.vars 
Vault password: 
Decryption successful
[root@ip-172-31-8-221 docker-image]# cat cred.vars 